# Hướng dẫn truy vấn dữ liệu thị giác dùng fiftyone

Đây là hướng dẫn dùng cho các đội tham dự AI Challenge 2023. Hướng dẫn này nhằm mục đích giới thiệu cho các đội một phương pháp cơ bản để truy vấn dữ liệu dựa trên thông tin BTC cung cấp và giới thiệu công cụ fiftyone để hỗ trợ đội thi đánh giá kết quả.

## Cài đặt ban đầu

Bạn cần cài đặt môi trường để chạy được notebook này trên máy tính cá nhân của bạn. Hướng dẫn này không bao gồm phần cài đặt môi trường. Khuyến nghị: các bạn có thể cài đặt [Anaconda](https://docs.anaconda.com/free/anaconda/install/windows/).

## Cài đặt các thư viện FiftyOne và PyTorch
Hướng dẫn này dùng fiftyone là công cụ để trực quan dữ liệu và pytorch là backend chính cho các thuật toán máy học.

### Lưu ý: Đối với các bạn dùng Windows nên dùng bản fiftyone **v0.21.4**, không nên dùng bản mới nhất!

Load dữ liệu keyframe từ thư mục chứa keyframe. Mỗi ảnh và thông tin đi kèm sau này sẽ được lưu trữ trong một Sample. Tất cả các Sample được lưu trong Dataset.

In [ ]:
! pip install fiftyone==0.21.4
! pip install torch torchvision torchaudio

In [1]:
import fiftyone as fo
import fiftyone.brain as fob
import numpy as np
from glob import glob
import json
import os



Load dữ liệu keyframe từ thư mục chứa keyframe. Trong hướng dẫn này tất cả các file Keyframes_L*.zip được giải nén vào thư mục `D:\AIC\Keyframes`. Mỗi ảnh và thông tin đi kèm sau này sẽ được lưu trữ trong một `Sample`. Tất cả các `Sample` được lưu trong `Dataset`.

In [3]:
dataset = fo.Dataset.from_images_dir('original_data/keyframes', name=None, tags=None, recursive=True)

 100% |███████████████| 7658/7658 [1.7s elapsed, 0s remaining, 4.6K samples/s]         


Sau khi dữ liệu đã load lên xong. Bạn có thể truy cập vào đường vào ứng dụng web của fiftyone từ [http://localhost:5151](http://localhost:5151)

In [20]:
session = fo.launch_app(dataset, auto=False)


Could not connect session, trying again in 10 seconds

Session launched. Run `session.show()` to open the App in a cell output.

Could not connect session, trying again in 10 seconds


Could not connect session, trying again in 10 seconds



In [ ]:
session.show()

Hoặc bạn có thể chạy cell bên dưới để mở tab mới cho ứng dụng web fiftyone

In [19]:
session.open_tab()

<IPython.core.display.Javascript object>

### Trích xuất thêm thông tin tên của video và frameid
Thông tin `video` và `frameid` sẽ được lấy từ tên của tập tin keyframe.

In [4]:
for sample in dataset:
    _, sample['video'], sample['frameid'] = sample['filepath'][:-4].rsplit('/', 2)
    sample.save()

Bạn có thể xem `Sample` đầu tiên của `Dataset` bằng lệnh sau:

In [8]:
print(dataset.first())

<Sample: {
    'id': '64ed6801c353dcf468117efa',
    'media_type': 'image',
    'filepath': '/Users/khanhmac/Desktop/hcmai-2023/baseline/data/keyframes/L01_V001/0001.jpg',
    'tags': [],
    'metadata': None,
}>


### Thêm thông tin kết quả của object detection.

Bước này có thể tốn của bạn nhiều thời gian để đọc hết tất cả các dữ liệu về object detection. Bạn có thể bỏ qua cell này và chạy cell này sau nếu muốn thử thêm các thông tin về vector CLIP embedding trước.

In [6]:
for sample in dataset:
    object_path = f"original_data/objects/{sample['video']}/{sample['frameid']}.json"
    with open(object_path) as jsonfile:
        det_data = json.load(jsonfile)
    detections = []
    for cls, box, score in zip(det_data['detection_class_entities'], det_data['detection_boxes'], det_data['detection_scores']):
        # Convert to [top-left-x, top-left-y, width, height]
        boxf = [float(box[1]), float(box[0]), float(box[3]) - float(box[1]), float(box[2]) - float(box[0])]
        scoref = float(score)

        # Only add objects with confidence > 0.4
        if scoref > 0.4:
            detections.append(
                fo.Detection(
                    label=cls,
                    bounding_box= boxf,
                    confidence=float(score)
                )
            )
    sample["object_faster_rcnn"] = fo.Detections(detections=detections)
    sample.save()


### Thêm thông tin CLIP embedding.

In [7]:
all_keyframe = glob('original_data/keyframes/*/*.jpg')
video_keyframe_dict = {}
all_video = glob('original_data/keyframes/*')
all_video = [v.rsplit('/',1)[-1] for v in all_video]

Đọc thông tin clip embedding được cung cấp.

Lưu ý: Các bạn cần tải đúng bản CLIP embedding từ model **CLIP ViT-B/32**

Tạo dictionary `video_keyframe_dict` với `video_keyframe_dict[video]` thông tin danh sách `keyframe` của `video`

In [8]:
for kf in all_keyframe:
    _, vid, kf = kf[:-4].rsplit('/',2)
    if vid not in video_keyframe_dict.keys():
        video_keyframe_dict[vid] = [kf]
    else:
        video_keyframe_dict[vid].append(kf)

Do thông tin vector CLIP embedding được cung cấp được lưu theo từng video nhầm mục đích tối ưu thời gian đọc dữ liệu. Cần sort lại danh sách `keyframe` của từng `video` để đảm bảo thứ tự đọc đúng với vector embedding được cung cấp.

In [9]:
for k,v in video_keyframe_dict.items():
    video_keyframe_dict[k] = sorted(v)

Tạo dictionary `embedding_dict` với `embedding_dict[video][keyframe]` lưu thông tin vector CLIP embedding của `keyframe` trong `video` tương ứng

In [11]:
embedding_dict = {}
for v in all_video:
    clip_path = f'original_data/clip-features-vit-b32/{v}.npy'
    a = np.load(clip_path)
    embedding_dict[v] = {}
    for i,k in enumerate(video_keyframe_dict[v]):
        embedding_dict[v][k] = a[i]


Tạo danh sách `clip_embedding` ứng với danh sách `sample` trong `dataset`.

In [12]:
clip_embeddings = []
for sample in dataset:
    clip_embedding = embedding_dict[sample['video']][sample['frameid']]
    clip_embeddings.append(clip_embedding)


temp_sc = np.array(clip_embeddings)
print(temp_sc.shape)

(7658, 512)


In [13]:
fob.compute_similarity(
    dataset,
    model="clip-vit-base32-torch",      # store model's name for future use
    embeddings=clip_embeddings,          # precomputed image embeddings
    brain_key="img_sim",
)

 100% |██████|    2.6Gb/2.6Gb [2.9m elapsed, 0s remaining, 21.2Mb/s]      
 100% |█████|   10.4Mb/10.4Mb [224.3ms elapsed, 0s remaining, 46.1Mb/s]      


## Từ đây các bạn có thể thử các tính năng search, filter trên ứng dụng fiftyone.

In [17]:
!pip install umap-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 1.6 MB/s eta 0:00:00 MB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for numba>=0.49 from https://files.pythonhosted.org/packages/d2/1c/4f3e6cb644811a8e6a3e0a8dccf1457ad05ee238c507a990521fb3c92acf/numba-0.57.1-cp38-cp38-macosx_10_9_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 3.9 MB/s eta 0:00:00m eta 0:00:010:00:01
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for tqdm from https://files.pythonhosted.org/packages/00/e5/f12a80907d0884e6dff9c16d0c0114d81b8cd07dc3ae54c5e962cc83037e/tqdm-4.66.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.1 MB/s eta 0:00:00
  Obtaining dependency information for llvmlite<0.41,>=0.40.0dev0 from https://files.pythonhosted.org/packages/b8/43/50bc2dbce04b020d635438f9cb026c56bfd3a01b40a9d1d2577521a25239/llvmlite-0.40.1-cp38-cp38-macosx_10_9_x86_64.whl.meta

In [18]:
# Bạn cần phải cài version umap-learn hỗ trợ.
# fob.compute_visualization(
#     dataset,
#     embeddings=clip_embeddings,
#     brain_key="img_viz"
# )


Generating visualization...


/Users/khanhmac/miniconda3/envs/py38/lib/python3.8/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/khanhmac/miniconda3/envs/py38/lib/python3.8/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/khanhmac/miniconda3/envs/py38/lib/python3.8/site-packages/uma

UMAP( verbose=True)
Wed Aug 30 06:11:41 2023 Construct fuzzy simplicial set
Wed Aug 30 06:11:41 2023 Finding Nearest Neighbors
Wed Aug 30 06:11:41 2023 Building RP forest with 9 trees
Wed Aug 30 06:11:48 2023 NN descent for 13 iterations
	 1  /  13
	 2  /  13
	 3  /  13
	 4  /  13
	 5  /  13
	Stopping threshold met -- exiting after 5 iterations
Wed Aug 30 06:12:06 2023 Finished Nearest Neighbor Search
Wed Aug 30 06:12:09 2023 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

Wed Aug 30 06:12:18 2023 Finished embedding
